<a href="https://colab.research.google.com/github/Sandhiya072005/DATA-ANALYSIS/blob/main/Intruze_Email_Seg_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import imaplib
import email
from email.header import decode_header
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tkinter as tk


In [ ]:
def fetch_emails():
    """
    Fetch the latest unseen email from the inbox.
    Returns:
        sender, subject, body of the email
    """

    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login("colloquium.arjun@gmail.com", "Symposium2023")
    mail.select("inbox")

    status, messages = mail.search(None, 'UNSEEN')
    email_ids = messages[0].split()

    # Processing the latest unseen email
    for email_id in email_ids:
        status, msg_data = mail.fetch(email_id, "(RFC822)")
        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1])
                subject, encoding = decode_header(msg["Subject"])[0]
                if isinstance(subject, bytes):
                    subject = subject.decode(encoding if encoding else "utf-8")
                sender = msg.get("From")
                payload = msg.get_payload(decode=True).decode() if msg.is_multipart() else msg.get_payload()
                return sender, subject, payload
    return None


In [ ]:
def extract_urls(text):
    """
    Extracts all URLs from the given email text using regex.
    Args:
        text: Email body as a string.
    Returns:
        List of URLs found in the text.
    """
    url_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    urls = re.findall(url_pattern, text)
    return urls

def is_phishing_url(url):
    """
    Checks if a URL is suspicious based on a few heuristic patterns.
    Args:
        url: URL string to check.
    Returns:
        True if the URL is suspicious, False otherwise.
    """
    suspicious_patterns = ['.xyz', '.ru', 'login', 'account', 'verify', 'bank']
    return any(pattern in url for pattern in suspicious_patterns)


In [ ]:
emails = [
    # Phishing emails
    "Your account has been compromised. Please log in here to change your password immediately: http://fake-phishing-site.com", # 1
    "Urgent: You have a pending transaction in your account. Please verify your identity here: http://verify-now.com", # 1
    "Dear customer, we have detected suspicious activity on your bank account. Click here to secure your account: http://bank-verify.com", # 1
    "You've won a $5000 Amazon gift card! Click to claim it now: http://phishing-claim.com", # 1
    "Important: Your PayPal account has been frozen due to suspicious activity. Please verify your identity here: http://paypal-security.com", # 1
    "Security Alert: Your account is under review. Please verify your identity immediately to avoid restrictions: http://account-review.com", # 1
    "Dear user, your login details have been compromised. Change your password here: http://login-change.com", # 1
    "Congratulations! You’ve been selected for a free iPhone. Claim it now by clicking here: http://phishing-phone-claim.com", # 1
    "Act fast! Your subscription is expiring. Renew now for a 50% discount: http://phishing-renew.com", # 1
    "Attention! Your bank account has been locked due to security concerns. Click here to unlock it: http://unlock-account.com", # 1

    # Legitimate emails
    "Hi John, Your Amazon order has shipped! Track your package here: http://amazon-tracking.com", # 0
    "Dear User, Your subscription has been renewed successfully. Thank you for being a valued member.", # 0
    "Reminder: Your monthly newsletter subscription is due for renewal next week. If you wish to continue, please visit our website.", # 0
    "Hello, Please find your invoice for last month's service attached. Let us know if you have any questions.", # 0
    "Your account has been successfully updated. If you did not request this change, please contact support immediately.", # 0
    "Hi, just wanted to confirm your upcoming meeting with Dr. Smith at 2:00 PM tomorrow. See you soon!", # 0
    "Your credit card statement is ready for review. Please log in to your account to view your transactions.", # 0
    "Dear Sarah, Your recent purchase from Best Buy has been processed. Thank you for your order!", # 0
    "Reminder: Your gym membership renewal is coming up next month. Click here for more details: http://gym-website.com", # 0
    "Hello Jane, Your package from UPS will be delivered tomorrow between 10 AM and 12 PM.", # 0

    # Phishing emails
    "Urgent: Your account is about to be suspended due to unpaid dues. Please click here to pay now: http://phishing-pay-now.com", # 1
    "You have a limited-time offer to claim a free iPad. Don’t miss out! Click here: http://fake-offer-ipad.com", # 1
    "Security Alert: Someone tried to log in to your Google account from an unrecognized device. Click here to secure it: http://google-security.com", # 1
    "You’ve been selected for a job offer! Send us your personal details to get started: http://job-offer-site.com", # 1
    "Get a 90% discount on your next purchase! Just click here to claim: http://phishing-discount.com", # 1

    # Legitimate emails
    "Dear team, Please review the attached document for the upcoming project deadline. Let me know if you have any questions.", # 0
    "Hello, Your annual subscription to Netflix will automatically renew next month. You can manage your subscription here: http://netflix-manage.com", # 0
    "Hi, here’s your monthly performance report. Let me know if you'd like to discuss it further. Cheers!", # 0
    "Your LinkedIn connection request from Jane Smith has been approved. You can now view her profile here: http://linkedin.com", # 0
    "Hi, your recent bank transfer was successfully processed. If you have any questions, please contact support.", # 0

    # More varied phishing emails
    "Click here to win a free luxury vacation to Paris! All you need to do is provide your payment details: http://vacation-prize.com", # 1
    "Security notice: Your Apple ID has been compromised. Reset your password immediately: http://apple-id-reset.com", # 1
    "Dear Valued Customer, your tax return has been processed. Please click here to confirm your details: http://tax-confirmation.com", # 1
    "Congratulations! You've been selected to receive a $1000 Amazon gift card. Claim it now by clicking here: http://gift-card-phish.com", # 1
    "Action required: Your Netflix subscription has been blocked. Click here to unlock it: http://netflix-unlock.com", # 1

    # More varied legitimate emails
    "Reminder: Your flight with Delta Airlines is scheduled for tomorrow. Please check in here: http://delta-checkin.com", # 0
    "Important: Your Apple subscription has been updated. Click here to view the new terms and conditions: http://apple.com", # 0
    "Hello, your password has been successfully updated. If you didn’t make this change, please click here: http://secure-account.com", # 0
    "Hi, your quarterly report is now available. Please review the attached document for the details.", # 0
    "Dear customer, thank you for booking your appointment with Dr. Lee. Your appointment is scheduled for Friday at 10 AM.", # 0
]

# Manually assigned labels: 1 for phishing, 0 for legitimate
labels = [
    1, 1, 1, 1, 1, 1, 1, 1, 1, 1,   # Phishing emails
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   # Legitimate emails
    1, 1, 1, 1, 1,                  # Phishing emails
    0, 0, 0, 0, 0,                  # Legitimate emails
    1, 1, 1, 1, 1,                  # Phishing emails
    0, 0, 0, 0, 0,                  # Legitimate emails
]

additional_emails = [
    # More phishing examples
    "Attention! Your bank account has been accessed from an unknown location. Confirm your identity here: http://bank-confirm.com",
    "Your Amazon account has been suspended. Please log in to verify your information: http://fake-amazon.com",
    "You've been chosen for a special prize! Click here to receive a $1000 voucher: http://voucher-prize.com",
    "Your account has suspicious activity. Secure it now by clicking here: http://secure-account-phish.com",
    "Urgent! Verify your account information to prevent suspension: http://urgent-verify.com",
    "Click here to confirm your $500 cash prize! Just verify your details: http://cash-prize.com",
    "Suspicious login detected in your bank account. Secure it now: http://secure-bank.com",
    "Your credit card was used for an unauthorized purchase. Review your statement here: http://fake-statement.com",
    "Congratulations! You've won a luxury cruise. Claim your prize by entering your details: http://cruise-claim.com",
    "Your Netflix account has been blocked due to suspicious login. Click here to verify: http://fake-netflix.com",

    # More legitimate examples
    "Hello, this is a reminder of your dental appointment scheduled for next week. Please confirm your attendance.",
    "Dear user, your password has been changed successfully. If this wasn't you, contact support immediately.",
    "Your monthly statement is ready. Log into your account to view it securely.",
    "Reminder: Your car service is scheduled for Friday. Let us know if you need to reschedule.",
    "Hi there! Thank you for subscribing to our newsletter. We're excited to share updates with you.",
    "Dear customer, your annual subscription has been renewed. Contact support if you have any questions.",
    "Hi Alex, your recent flight reservation has been confirmed. We'll send you updates as the date approaches.",
    "Your package from FedEx will arrive tomorrow. Track your delivery for updates on the arrival time.",
    "Dear valued member, thank you for renewing your membership. We’re excited to have you with us for another year.",
    "Hello, please find your recent order confirmation attached. Thank you for shopping with us!"
]

# Corresponding labels: 1 for phishing, 0 for legitimate
additional_labels = [
    1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  # Additional phishing emails
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0   # Additional legitimate emails
]

# Combine original and additional data
emails.extend(additional_emails)
labels.extend(additional_labels)

# Additional phishing emails
additional_emails_phishing = [
    "Warning: Unusual activity detected in your Facebook account. Log in to verify: http://facebook-security-check.com",
    "Congratulations! You've been randomly selected for a $500 reward. Claim your prize here: http://fake-reward.com",
    "Dear customer, your recent purchase was flagged for potential fraud. Confirm your identity: http://purchase-confirm.com",
    "Urgent: Please update your billing information to prevent service interruption: http://fake-billing-update.com",
    "Security Update: We detected unauthorized login attempts. Verify your account: http://login-verification.com",
    "You have received a tax refund. Click here to claim it: http://fake-tax-claim.com",
    "We've noticed suspicious logins on your account. Secure it by clicking here: http://suspicious-login.com",
    "Act now! Your payment could not be processed. Update your information here: http://payment-fail.com",
    "Your social security number has been compromised. Secure your account here: http://secure-ssn.com",
    "Dear user, your Dropbox files will be deleted if not verified. Confirm your account here: http://fake-dropbox.com"
]

# Additional legitimate emails
additional_emails_legitimate = [
    "Hi Alex, your Amazon order has shipped! You can track your package here: http://amazon-tracking.com",
    "Your monthly bank statement is ready. Log in to review your latest transactions securely.",
    "Hello team, please find attached the project proposal draft for next week’s meeting.",
    "Reminder: Your dentist appointment is on Friday at 10 AM. Let us know if you need to reschedule.",
    "Your subscription renewal was successful. Thank you for being a valued customer.",
    "Dear customer, your electricity bill for the month is now available. Pay by the due date to avoid penalties.",
    "Thank you for your recent donation. We appreciate your support in helping us make a difference!",
    "Hello, your scheduled maintenance is coming up soon. Reply if you need to change the date.",
    "Hi Rachel, we hope you're enjoying your membership! Here’s an exclusive offer just for you.",
    "Your flight reservation has been confirmed. Check your email for the e-ticket and other travel details."
]

# Labels for the additional emails
additional_labels_phishing = [1] * len(additional_emails_phishing)
additional_labels_legitimate = [0] * len(additional_emails_legitimate)

# Combine all additional emails and labels
additional_emails = additional_emails_phishing + additional_emails_legitimate
additional_labels = additional_labels_phishing + additional_labels_legitimate

# Combine original emails and additional emails into a single dataset
all_emails = emails + additional_emails
all_labels = labels + additional_labels
# Step 1: Convert text data into numerical features using CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2))
X = vectorizer.fit_transform(emails)
y = labels


# Step 2: Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


# Step 3: Train a Logistic Regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)  # Train on the data


# Step 4: Evaluate the model on the test set
from sklearn.metrics import accuracy_score
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)


print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 93.33%


In [ ]:
def detect_phishing_in_email(sender, subject, body):
    """
    Detects phishing in an email by analyzing its URLs and content.

    Args:
        sender (str): Sender of the email.
        subject (str): Subject of the email.
        body (str): Body text of the email.

    Returns:
        Tuple (int, list): Prediction (1 if phishing, 0 if legitimate) and list of detected phishing URLs.
    """

    # Step 1: Extract URLs from the email body
    urls = extract_urls(body)

    # Step 2: Identify phishing URLs from extracted URLs
    phishing_urls = [url for url in urls if is_phishing_url(url)]

    # Step 3: Analyze email content using the trained model
    features = vectorizer.transform([body])  # Convert email body text into features
    prediction = model.predict(features)[0]  # Predict if the email is phishing or legitimate

    return prediction, phishing_urls




In [ ]:
import imaplib
import email
from email.header import decode_header
import re
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Function to connect to Gmail and fetch emails
def fetch_emails_from_gmail(username, password, n=5):
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(username, password)
    mail.select("inbox")
    status, messages = mail.search(None, "ALL")
    email_ids = messages[0].split()
    email_data = []
    for email_id in email_ids[-n:]:
        status, msg_data = mail.fetch(email_id, "(RFC822)")
        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1])
                subject, encoding = decode_header(msg["Subject"])[0]
                if isinstance(subject, bytes):
                    subject = subject.decode(encoding if encoding else "utf-8")
                sender = msg.get("From")
                if msg.is_multipart():
                    for part in msg.walk():
                        content_type = part.get_content_type()
                        if content_type == "text/plain":
                            body = part.get_payload(decode=True).decode()
                            email_data.append((sender, subject, body))
                else:
                    body = msg.get_payload(decode=True).decode()
                    email_data.append((sender, subject, body))
    mail.logout()
    return email_data


# Helper functions for processing and displaying email content
def extract_urls(text):
    urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    return urls


def classify_email(text, vectorizer, model):
    text_features = vectorizer.transform([text])
    return model.predict(text_features)[0]


# Header with title styling
header = widgets.HTML(
    value="<h2 style='color: white; font-family: Arial, sans-serif;'>✉️ Email Phishing Detector</h2>",
    layout=widgets.Layout(
        background='linear-gradient(90deg, #4e54c8, #8f94fb)',
        padding='15px',
        width='100%',
        border_radius='5px'
    )
)


# Widgets for displaying status, results, URLs, and email content
status_label = widgets.Label(value="Listening for new emails...", style={'description_width': 'initial'})
result_label = widgets.HTML(value="<b>Result:</b> Waiting for email...")
urls_label = widgets.Textarea(
    value="Suspicious URLs will appear here.",
    layout=widgets.Layout(width='100%', height='80px', border='1px solid #ddd', padding='10px', font_family='Arial')
)
email_content_label = widgets.Textarea(
    value="Email content will appear here.",
    layout=widgets.Layout(width='100%', height='300px', border='1px solid #ddd', padding='10px', font_family='Arial')
)


# Buttons with customized colors
fetch_button = widgets.Button(
    description="Fetch New Email",
    button_style='',
    icon="envelope",
    layout=widgets.Layout(width='150px', height='40px', background_color='#556b2f', color='white'),
    style={'button_color': '#556b2f', 'font_weight': 'bold'}
)


clear_button = widgets.Button(
    description="Clear Display",
    button_style='',
    icon="trash",
    layout=widgets.Layout(width='150px', height='40px', background_color='#fb8b24', color='white'),
    style={'button_color': '#fb8b24', 'font_weight': 'bold'}
)


# Sidebar with security tips box
security_tips = widgets.HTML(
    value="<div style='color:white; font-family: Arial;'>"
          "<h4 style='margin-top:10px;'>Email Security Tips</h4>"
          "<h5>* Don't click on suspicious links.</h5>"
          "<h5>* Verify the sender.</h5>"
          "<h5>* Enable two-factor authentication.</h5>",
    layout=widgets.Layout(
        background='linear-gradient(120deg, #6e7eff, #b3c0ff)',
        padding='15px',
        border_radius='10px',
        width='90%',
        margin='10px 0 0 0'
    )
)


# Sidebar layout
sidebar = widgets.VBox(
    [
        widgets.HTML("<h3 style='color:white; font-family: Arial;'>Options</h3>"),
        fetch_button,
        clear_button,
        widgets.HTML("<p style='color:white; margin-top:15px;'>Quick Links:</p>"),
        widgets.HTML("<a href='https://support.google.com/mail/' style='color:lightblue;'>Google Support</a>"),
        widgets.HTML("<a href='https://haveibeenpwned.com/' style='color:lightblue;'>Check Data Breach</a>"),
        security_tips  # Added security tips box with updated format
    ],
    layout=widgets.Layout(
        width='180px',
        background='linear-gradient(120deg, #6e7eff, #b3c0ff)',
        color='white',
        padding='20px',
        border_radius='10px',
        box_shadow='0px 0px 10px rgba(0, 0, 0, 0.1)',
    )
)


# Main content layout
main_content = widgets.VBox(
    [
        status_label,
        widgets.HTML("<hr style='border: none; border-top: 1px solid #ddd;'>"),
        result_label,
        widgets.HTML("<hr style='border: none; border-top: 1px solid #ddd;'>"),
        widgets.HTML("<h4>Suspicious URLs:</h4>"),
        urls_label,
        widgets.HTML("<hr style='border: none; border-top: 1px solid #ddd;'>"),
        widgets.HTML("<h4>Email Content:</h4>"),
        email_content_label
    ],
    layout=widgets.Layout(
        padding='20px',
        border_radius='10px',
        background='linear-gradient(90deg, #f1f1f1, #e0e6ff)',
        width='75%',
        box_shadow='0px 0px 10px rgba(0, 0, 0, 0.1)',
    )
)


# Layout combining sidebar and main content
app_layout = widgets.HBox([sidebar, main_content])


# Display the header and layout
display(header, app_layout)


# Function to fetch and analyze the latest email
def on_fetch_button_click(b):
    username = "colloquium.arjun@gmail.com"  # Your email
    password = "umov tvod pbxm yslx"  # Your app password


    email_data = fetch_emails_from_gmail(username, password, n=1)
    if email_data:
        sender, subject, body = email_data[0]
        urls = extract_urls(body)
        result = classify_email(body, vectorizer, model)
        result_text = "Phishing Detected" if result == 1 else "Legitimate Email"
        result_label.value = f"<b>Result:</b> {result_text}"
        urls_label.value = "\n".join(urls) if urls else "No suspicious URLs detected."
        email_content_label.value = f"From: {sender}\nSubject: {subject}\n\n{body}"


# Function to clear displayed data
def on_clear_button_click(b):
    result_label.value = "<b>Result:</b> Waiting for email..."
    urls_label.value = "Suspicious URLs will appear here."
    email_content_label.value = "Email content will appear here."


# Bind button actions
fetch_button.on_click(on_fetch_button_click)
clear_button.on_click(on_clear_button_click)

emails = [

    # Phishing emails (labeled as 1)
    "Your account has been compromised. Please log in here to change your password immediately: http://fake-phishing-site.com", # 1
    "Urgent: You have a pending transaction in your account. Please verify your identity here: http://verify-now.com", # 1
    "Dear customer, we have detected suspicious activity on your bank account. Click here to secure your account: http://bank-verify.com", # 1
    "You've won a $5000 Amazon gift card! Click to claim it now: http://phishing-claim.com", # 1
    "Important: Your PayPal account has been frozen due to suspicious activity. Please verify your identity here: http://paypal-security.com", # 1
    "Security Alert: Your account is under review. Please verify your identity immediately to avoid restrictions: http://account-review.com", # 1
    "Dear user, your login details have been compromised. Change your password here: http://login-change.com", # 1
    "Congratulations! You’ve been selected for a free iPhone. Claim it now by clicking here: http://phishing-phone-claim.com", # 1
    "Act fast! Your subscription is expiring. Renew now for a 50% discount: http://phishing-renew.com", # 1
    "Attention! Your bank account has been locked due to security concerns. Click here to unlock it: http://unlock-account.com", # 1

    # Legitimate emails (labeled as 0)
    "Hi John, Your Amazon order has shipped! Track your package here: http://amazon-tracking.com", # 0
    "Dear User, Your subscription has been renewed successfully. Thank you for being a valued member.", # 0
    "Reminder: Your monthly newsletter subscription is due for renewal next week. If you wish to continue, please visit our website.", # 0
    "Hello, Please find your invoice for last month's service attached. Let us know if you have any questions.", # 0
    "Your account has been successfully updated. If you did not request this change, please contact support immediately.", # 0
    "Hi, just wanted to confirm your upcoming meeting with Dr. Smith at 2:00 PM tomorrow. See you soon!", # 0
    "Your credit card statement is ready for review. Please log in to your account to view your transactions.", # 0
    "Dear Sarah, Your recent purchase from Best Buy has been processed. Thank you for your order!", # 0
    "Reminder: Your gym membership renewal is coming up next month. Click here for more details: http://gym-website.com", # 0
    "Hello Jane, Your package from UPS will be delivered tomorrow between 10 AM and 12 PM.", # 0

    # Phishing emails (labeled as 1)
    "Urgent: Your account is about to be suspended due to unpaid dues. Please click here to pay now: http://phishing-pay-now.com", # 1
    "You have a limited-time offer to claim a free iPad. Don’t miss out! Click here: http://fake-offer-ipad.com", # 1
    "Security Alert: Someone tried to log in to your Google account from an unrecognized device. Click here to secure it: http://google-security.com", # 1
    "You’ve been selected for a job offer! Send us your personal details to get started: http://job-offer-site.com", # 1
    "Get a 90% discount on your next purchase! Just click here to claim: http://phishing-discount.com", # 1

    # Legitimate emails (labeled as 0)
    "Dear team, Please review the attached document for the upcoming project deadline. Let me know if you have any questions.", # 0
    "Hello, Your annual subscription to Netflix will automatically renew next month. You can manage your subscription here: http://netflix-manage.com", # 0
    "Hi, here’s your monthly performance report. Let me know if you'd like to discuss it further. Cheers!", # 0
    "Your LinkedIn connection request from Jane Smith has been approved. You can now view her profile here: http://linkedin.com", # 0
    "Hi, your recent bank transfer was successfully processed. If you have any questions, please contact support.", # 0

    # More varied phishing emails (labeled as 1)
    "Click here to win a free luxury vacation to Paris! All you need to do is provide your payment details: http://vacation-prize.com", # 1
    "Security notice: Your Apple ID has been compromised. Reset your password immediately: http://apple-id-reset.com", # 1
    "Dear Valued Customer, your tax return has been processed. Please click here to confirm your details: http://tax-confirmation.com", # 1
    "Congratulations! You've been selected to receive a $1000 Amazon gift card. Claim it now by clicking here: http://gift-card-phish.com", # 1
    "Action required: Your Netflix subscription has been blocked. Click here to unlock it: http://netflix-unlock.com", # 1

    # More varied legitimate emails (labeled as 0)
    "Reminder: Your flight with Delta Airlines is scheduled for tomorrow. Please check in here: http://delta-checkin.com", # 0
    "Important: Your Apple subscription has been updated. Click here to view the new terms and conditions: http://apple.com", # 0
    "Hello, your password has been successfully updated. If you didn’t make this change, please click here: http://secure-account.com", # 0
    "Hi, your quarterly report is now available. Please review the attached document for the details.", # 0
    "Dear customer, thank you for booking your appointment with Dr. Lee. Your appointment is scheduled for Friday at 10 AM.", # 0
]

# Manually labeled data
labels = [
    1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  # Phishing emails
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  # Legitimate emails
    1, 1, 1, 1, 1,                  # Phishing emails
    0, 0, 0, 0, 0,  # Legitimate emails
    1, 1, 1, 1, 1,                  # Phishing emails
    0, 0, 0, 0, 0,                  # Legitimate emails
]  # 1 for phishing, 0 for legitimate
additional_emails = [
    # More phishing examples
    "Attention! Your bank account has been accessed from an unknown location. Confirm your identity here: http://bank-confirm.com",
    "Your Amazon account has been suspended. Please log in to verify your information: http://fake-amazon.com",
    "You've been chosen for a special prize! Click here to receive a $1000 voucher: http://voucher-prize.com",
    "Your account has suspicious activity. Secure it now by clicking here: http://secure-account-phish.com",
    "Urgent! Verify your account information to prevent suspension: http://urgent-verify.com",
    "Click here to confirm your $500 cash prize! Just verify your details: http://cash-prize.com",
    "Suspicious login detected in your bank account. Secure it now: http://secure-bank.com",
    "Your credit card was used for an unauthorized purchase. Review your statement here: http://fake-statement.com",
    "Congratulations! You've won a luxury cruise. Claim your prize by entering your details: http://cruise-claim.com",
    "Your Netflix account has been blocked due to suspicious login. Click here to verify: http://fake-netflix.com",

    # More legitimate examples
    "Hello, this is a reminder of your dental appointment scheduled for next week. Please confirm your attendance.",
    "Dear user, your password has been changed successfully. If this wasn't you, contact support immediately.",
    "Your monthly statement is ready. Log into your account to view it securely.",
    "Reminder: Your car service is scheduled for Friday. Let us know if you need to reschedule.",
    "Hi there! Thank you for subscribing to our newsletter. We're excited to share updates with you.",
    "Dear customer, your annual subscription has been renewed. Contact support if you have any questions.",
    "Hi Alex, your recent flight reservation has been confirmed. We'll send you updates as the date approaches.",
    "Your package from FedEx will arrive tomorrow. Track your delivery for updates on the arrival time.",
    "Dear valued member, thank you for renewing your membership. We’re excited to have you with us for another year.",
    "Hello, please find your recent order confirmation attached. Thank you for shopping with us!"
]

# Corresponding labels: 1 for phishing, 0 for legitimate
additional_labels = [
    1, 1, 1, 1, 1, 1, 1, 1, 1, 1,  # Additional phishing emails
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0   # Additional legitimate emails
]

# Combine original and additional data
emails.extend(additional_emails)
labels.extend(additional_labels)

# Additional phishing emails
additional_emails_phishing = [
    "Warning: Unusual activity detected in your Facebook account. Log in to verify: http://facebook-security-check.com",
    "Congratulations! You've been randomly selected for a $500 reward. Claim your prize here: http://fake-reward.com",
    "Dear customer, your recent purchase was flagged for potential fraud. Confirm your identity: http://purchase-confirm.com",
    "Urgent: Please update your billing information to prevent service interruption: http://fake-billing-update.com",
    "Security Update: We detected unauthorized login attempts. Verify your account: http://login-verification.com",
    "You have received a tax refund. Click here to claim it: http://fake-tax-claim.com",
    "We've noticed suspicious logins on your account. Secure it by clicking here: http://suspicious-login.com",
    "Act now! Your payment could not be processed. Update your information here: http://payment-fail.com",
    "Your social security number has been compromised. Secure your account here: http://secure-ssn.com",
    "Dear user, your Dropbox files will be deleted if not verified. Confirm your account here: http://fake-dropbox.com"
]

# Additional legitimate emails
additional_emails_legitimate = [
    "Hi Alex, your Amazon order has shipped! You can track your package here: http://amazon-tracking.com",
    "Your monthly bank statement is ready. Log in to review your latest transactions securely.",
    "Hello team, please find attached the project proposal draft for next week’s meeting.",
    "Reminder: Your dentist appointment is on Friday at 10 AM. Let us know if you need to reschedule.",
    "Your subscription renewal was successful. Thank you for being a valued customer.",
    "Dear customer, your electricity bill for the month is now available. Pay by the due date to avoid penalties.",
    "Thank you for your recent donation. We appreciate your support in helping us make a difference!",
    "Hello, your scheduled maintenance is coming up soon. Reply if you need to change the date.",
    "Hi Rachel, we hope you're enjoying your membership! Here’s an exclusive offer just for you.",
    "Your flight reservation has been confirmed. Check your email for the e-ticket and other travel details."
]

# Labels for the additional emails
additional_labels_phishing = [1] * len(additional_emails_phishing)
additional_labels_legitimate = [0] * len(additional_emails_legitimate)

# Combine all additional emails and labels
additional_emails = additional_emails_phishing + additional_emails_legitimate
additional_labels = additional_labels_phishing + additional_labels_legitimate

# Combine original emails and additional emails into a single dataset
all_emails = emails + additional_emails
all_labels = labels + additional_labels


# Convert text data into numerical features using CountVectorizer
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2))
X = vectorizer.fit_transform(emails)  # Convert emails into features
y = labels  # Manually labeled data

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)  # Train on the data

# Now, you can run the GUI, and when you click "Analyze New Email", it will fetch and analyze real emails from your inbox.


HTML(value="<h2 style='color: white; font-family: Arial, sans-serif;'>✉️ Email Phishing Detector</h2>", layout…

LogisticRegression(max_iter=1000)